## 모듈 import

In [1]:
#모듈 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pycaret.classification  import *

# feature selection
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")

# permutation importance
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import eli5 
from eli5.sklearn import PermutationImportance

#Scailing
from sklearn.preprocessing import PowerTransformer

#Label Encoding
from sklearn.preprocessing import LabelEncoder

# 상관관계 시각화
#matplotlib 한글깨짐 지원
import platform
from matplotlib import font_manager, rc
path = "c:/Windows/Fonts/malgun.ttf"
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...')
rc('axes', unicode_minus=False)  
import seaborn as sns #EDA

In [2]:
#삡운드
import winsound as sd
# 삡싸운드
def beepsound():
    fr = 2000   # range : 37 ~ 32767
    du = 1000 # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)

## data 불러오기

In [3]:
# 데이터가 저장된 경로 설정
data_path = '../daplatformers2021/'

# 데이터 불러오기
train = pd.read_csv(data_path + 'trainset.csv')
test = pd.read_csv(data_path + 'testset_final.csv')
submission = pd.read_csv(data_path + 'sample_submission_final.csv')

In [4]:
train = train.drop(columns=['매니저최초가입일', '매니저최초서비스일', '매니저주소'])

### 전처리

In [1]:
data = pd.concat([train,test]).reset_index(drop=True)

- 결측치 채움

In [6]:
### 결측치 제거

# 결측치 0으로 채움
data['청소교육이수여부'] = data['청소교육이수여부'].fillna(0)
data['CS교육이수여부'] = data['CS교육이수여부'].fillna(0)
data['부재중여부'] = data['부재중여부'].fillna(0)
# 결측치 최빈값으로 채움
data['매니저이동방법'] = data['매니저이동방법'].fillna(data['매니저이동방법'].mode()[0])
data['매니저사용휴대폰'] = data['매니저사용휴대폰'].fillna(data['매니저사용휴대폰'].mode()[0])
data['고객가입일'] = data['고객가입일'].fillna(data['고객가입일'].mode()[0]) #데이터 바뀌고 test에 생긴 에러
data['결재형태'] = data['결재형태'].fillna(data['결재형태'].mode()[0]) #데이터 바뀌고 test에 생긴 에러

data['접수시각'] = data['접수시각'].fillna(data['접수시각'].mode()[0])
data['반려동물'] = data['반려동물'].fillna('없음')
# 결측치 판단하여 없음으로 채움
data['우선청소'] = data['우선청소'].fillna('없음')

### 평수 평균 확인 -> 31.xx -> 30평대로 채움
# (10*len(data.query('평수 == "10평대"')) + 20*len(data.query('평수 == "20평대"')) + 
#  30*len(data.query('평수 == "30평대"')) + 40*len(data.query('평수 == "40평대이상"')))/ \
# (len(data.query('평수 == "10평대"'))+len(data.query('평수 == "20평대"'))+
#  len(data.query('평수 == "30평대"'))+len(data.query('평수 == "40평대이상"')))
data['평수'] = data['평수'].fillna('30평대')

In [7]:
#test와 train의 구조를 맞춰줌
data['매니저생년월일'] = data['매니저생년월일'].astype('str').apply(lambda x: x[0:4])

In [8]:
# 새로운 피처 만들기
data['매니저나이'] = data['매니저생년월일'].apply(lambda x: 2021-int(x))
data['매니저나이대'] = data['매니저나이'].apply(lambda x: f'{str(x)[0]}0대')
# 현재 월을 기준으로 가입한지 몇개월이 됐는지 계산
data['고객가입개월수_str'] = data['고객가입일'].apply(lambda x:str((2021-(int(x[0:4])+1))*12 + 12-int(x[5:7])+1 + 11) )
data['고객가입개월수_int'] = data['고객가입일'].apply(lambda x:(2021-(int(x[0:4])+1))*12 + 12-int(x[5:7])+1 + 11)
data['나이*개월수'] = data['매니저나이']*data['고객가입개월수_int']

#전체회차 / 고객가입개월수_int
data['한달평균사용횟수'] = data['전체회차'] / data['고객가입개월수_int']

# 서비스시간 피처 생성
data['서비스시간'] =  (pd.to_datetime(data['서비스시작시간'], format="%H:%M:%S") -\
                        pd.to_datetime(data['서비스종료시간'], format="%H:%M:%S")).astype('str')

def mon_cnt(x):
    x = int(x)
    if x < 12: return '1년미만'
    elif 12 <= x < 24: return '1년차'
    elif 24 <= x < 36: return '2년차'
    
data['고객가입년차'] = data['고객가입개월수_int'].apply(mon_cnt)
# 교육
data['교육'] = data['청소교육이수여부'] + data['CS교육이수여부']

In [14]:
# 저장해놓기
data_SEQ = data.SEQ
# 인코딩을 위하여 제거 / 이후 나중에 병합함
y_data = data['매칭성공여부']
data = data.drop(columns = ['SEQ','매칭성공여부'])

In [15]:
#라벨인코딩

#범주형변수 수동으로 분리
data[['장기서비스여부','기존고객여부','부재중여부','쿠폰사용여부','매니저성별',
         'CS교육이수여부','청소교육이수여부','부재중서비스가능여부','추천인여부']
     ] = data[['장기서비스여부','기존고객여부','부재중여부','쿠폰사용여부','매니저성별',
         'CS교육이수여부','청소교육이수여부','부재중서비스가능여부','추천인여부'
     ]].astype('str')

# 범주형 변수와 수치형 변수를 분리
cat_features = data.select_dtypes(include=['object']).columns.to_list()
num_features = data.select_dtypes(exclude='object').columns.to_list() 


#라벨인코딩
#위의 범주형 변수들을 라벨인코딩
for f in cat_features:
    data[f] = LabelEncoder().fit_transform(data[f])
    
#수치형변수 scailing
data[num_features] = PowerTransformer(standardize=True).fit_transform(data[num_features])

In [2]:
#위에서 떼어놨던 피처 병합
data = pd.concat([data_SEQ, data],axis=1)
data = pd.concat([data, y_data],axis=1)

- 상관관계를 시각화하여 확인

In [18]:
features = data.drop(columns=['SEQ', '매칭성공여부'])

In [1]:
# 아래의 코드를 통해 그래프를 그려 확인해 볼 수 있음 -> 보안상 실행X
plt.figure(figsize=(15,15))
sns.heatmap(data = data.corr(), annot=True, 
fmt = '.2f', linewidths=.5, cmap='Blues')

In [20]:
# 상관관계 행렬
correlationMatrix = data.corr()

In [21]:
# cutoff 기준점보다 높은 상관관계를 파악하는 함수
def high_correlated(x, cutoff):
    index_list = []
    for i in range(0, len(x)):
        for j in range(i+1, len(x)):
            if x.iloc[i][j] >= cutoff:
                index_list.append([x.columns[i], x.columns[j]])
    print(index_list)

# 함수 실행
high_correlated(correlationMatrix, 0.7)

[['접수일', '최초서비스일'], ['접수일', '서비스일자'], ['접수일', '고객가입일'], ['장기서비스여부', '전체회차'], ['최초서비스일', '서비스일자'], ['최초서비스일', '고객가입일'], ['서비스일자', '고객가입일'], ['CS교육이수여부', '청소교육이수여부'], ['매니저나이', '매니저나이대']]


In [22]:
#train과 test를 분리
train = data.iloc[:23009,:]
test = data.iloc[23009:,:]
test = test.drop(columns = ['매칭성공여부']).reset_index(drop=True)

### PermutationImportance

In [23]:
X_train = train.iloc[:,1:-1]
Y_train = train.iloc[:,-1]

In [2]:
## 아래의 코드를 통해 중요도가 높은 피처를 확인할 수 있음 -> 보안상 실행X
# 검증을 위해 train_test_split
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, shuffle=True, stratify=Y_train, random_state=42)
# 검증할 모델설정
my_model = RandomForestClassifier(n_estimators=100,
                                  random_state=0).fit(x_train, y_train)
# permutation importance 화긴
perm = PermutationImportance(my_model, scoring = "roc_auc", random_state = 42).fit(x_val, y_val) 
eli5.show_weights(perm,top=40, feature_names = x_val.columns.tolist())

# END